In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 42
BATCH_SIZE = 16

In [ ]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

# Create Datasets

In [ ]:
# Download latest version of data
image_dir = kagglehub.dataset_download("bloox2/fieldplant")
image_dir = Path(image_dir)
print("Path to dataset files:", image_dir)

# Create Models

In [ ]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape)
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [ ]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [ ]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_crossentropy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [ ]:
def print_model_info(model):
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:", compile_config['metrics'])
    print("Classifier layer activation function:", classifier_activation)
    print()
    

In [ ]:
def get_callbacks():
    cbs = [
        callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True, baseline=None, verbose=1),
        callbacks.ModelCheckpoint(filepath=f"{saved_models_dir}{model.name}.keras", save_best_only=True, monitor="val_loss", verbose=1, initial_value_threshold=None)]
    return cbs


# GET DATASETS

In [ ]:
def get_dataframe(filtered=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    return df   

In [ ]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [ ]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_dataset_for_performance(dataset, shuffle=False, batch_size=BATCH_SIZE):
    dataset = dataset.cache()
    if shuffle:
        dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
    dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [ ]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    for split in splits:
        img = split.filename
        labels = split.drop(columns=["filename"])
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        dataset = configure_dataset_for_performance(dataset)
        datasets.append(dataset)
    return tuple(datasets)

In [ ]:
def get_datasets(img_size, filtered=False, test_size=0.2):
    df = get_dataframe(filtered)
    splits = get_train_test_splits(df, filtered=filtered, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    return datasets

# CREATE MODELS

In [ ]:
def run_all(epochs=5):
    i = 0
    app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    #app_names = ["mobilenet_v2"]
    methodologies = ["multiclass", "multilabel"]
    methodologies = ["multiclass"]


    for filtered in [False, True]:
        for methodology in methodologies:
            if not filtered and (methodology == "binary"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            for filtered in [False, True]:
                df = get_dataframe(filtered=filtered)
                num_classes = len(df.columns[1:])
                
                for app_name in app_names:
                    i += 1
                    print("Model:", i)
                    print("Filtered dataset:", filtered)
                    print("Methodology:", methodology)
           
                    model = build_model(app_name, activation=activation, num_classes=num_classes)
                    img_size = model.input_shape[1:3]
                    ds = get_datasets(img_size, filtered=filtered)
                
                    # model.compile(
                    #     loss=loss,
                    #     optimizer=keras.optimizers.Adam(learning_rate=BASE_LR),
                    #     metrics=metrics
                    # )
                    # print_model_info(model)
        
                    # cbs = get_callbacks()
        
                    # history = model.fit(train_ds, validation_data=val_ds,
                    #                     epochs=epochs, callbacks=cbs)
        
                    

In [ ]:
model = run_all()